In [ ]:
import pandas as pd
import pyxirr
import yfinance as yf
import numpy as np
from enum import Enum

In [ ]:
INVEST_FILE = 'G:\My Drive\invest\investment_returns.xlsx'
STOCK_FILE = 'G:\My Drive\invest\stock_purchase.xlsx'
SHEETS_TO_SOURCE_MAP = {"Saxo - SGD": "saxo", 
                        "IBKR - SGD(U7470748)": "ibkr",
                         "IBKR endowment plan": "ibkr_endowment", 
                         "Tiger broker": "tiger", 
                         "philips - SGD": "philips",
                         "moomoo": "moomoo"}

In [ ]:
def remove_unnamed_columns(dfs: dict) -> None:
    for key in dfs.keys():
        dfs[key] = dfs[key].loc[:, ~dfs[key].columns.str.contains('^Unnamed')]

def filter_invalid_timestamps(dfs: dict):
    for key, df in dfs.items():
        df['parse_timestamp'] = pd.to_datetime(df['Date'], errors='coerce')
        invalid_rows = df[df['parse_timestamp'].isna()]
        df.drop(invalid_rows.index, inplace=True)
        df['date'] = df['parse_timestamp'].dt.date
        df.drop(columns=['parse_timestamp', 'Date'], inplace=True)
        dfs[key] = df

def parse_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
    df = df.drop(columns=['Date'])
    return df[['date'] + [col for col in df.columns if col != 'date']]


def rename_columns(dfs: dict):
    for key, df in dfs.items():
        df.columns = [col.lower() for col in df.columns]
        df = df[['date', 'source', 'amount', 'desc']]
        dfs[key] = df

def get_sp500_data(start_date: pd.Timestamp, end_date: pd.Timestamp):
    sp500_ticker = '^GSPC'
    sp500_data = yf.download(sp500_ticker, start=start_date, end=end_date + pd.Timedelta(days=1), progress=False, auto_adjust=True)
    assert sp500_data is not None
    sp500_data.columns = sp500_data.columns.get_level_values(0)
    sp500_data.columns.name = None
    sp500_data.index = pd.to_datetime(sp500_data.index, errors='coerce').map(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
    return sp500_data

def calculate_my_returns() -> pd.DataFrame:
    sheets = list(SHEETS_TO_SOURCE_MAP.keys())
    dfs = {sheet : pd.read_excel(INVEST_FILE, sheet_name=sheet) for sheet in sheets}
    for sheet, source in SHEETS_TO_SOURCE_MAP.items():
        dfs[sheet]["source"] = source
    
    remove_unnamed_columns(dfs)
    filter_invalid_timestamps(dfs)
    rename_columns(dfs)

    df = pd.concat(dfs.values(), axis=0)
    df.date = pd.to_datetime(df.date, errors='coerce')
    df.sort_values(by='date', ascending=True, inplace=True)

    df_curr = df[df['desc'] == 'current value'].copy()
    df_other = df[df['desc'] != 'current value'].copy()

    df_other['cumulative_investment'] = -df_other['amount'].cumsum()
    df_curr['cumulative_investment'] = df_other['cumulative_investment'].iloc[-1]
    
    idx = df_curr['date'].idxmax()
    amount_sum = df_curr['amount'].sum()
    df_curr = df_curr.loc[[idx],:].copy()
    df_curr['amount'] = amount_sum
    df_curr['source'] = 'all_sources'


    df = pd.concat([df_other, df_curr], axis=0).sort_values(by='date', ascending=True).reset_index(drop=True)

    return df


def calculate_sp500_returns(my_df: pd.DataFrame) -> pd.DataFrame:
    start_date = my_df['date'].min()
    end_date = my_df['date'].max()
    sp500_prices = get_sp500_data(start_date, end_date)
    sp500_prices.index = pd.to_datetime(sp500_prices.index, errors='coerce')

    transactions = []
    total_shares = 0.0
    for _, rows in my_df.iterrows():
        date = rows['date']
        amount = rows['amount']
        available_dates = sp500_prices.index
        closest_date = min(available_dates, key=lambda d: abs(d - date))
        px = sp500_prices.loc[closest_date, 'Close'].astype(float)
        if rows['desc'] == 'current value':
            transactions.append({
                'date': date.date(),
                'type': 'current value',
                'qty': total_shares,
                'px': px,
                'amount': round(px * total_shares, 1),
                'cum_qty': total_shares,
                'market_value': round(px * total_shares)
            })
        elif amount < 0:
            shares_bought = abs(amount) / px
            total_shares += shares_bought
            transactions.append({
                'date': date.date(),
                'type': 'buy',
                'qty': shares_bought,
                'px': px,
                'amount': amount,
                'cum_qty': total_shares,
                'market_value': px * total_shares
            })
        else:
            shares_sold = amount / px
            total_shares -= shares_sold
            transactions.append({
                'date': date.date(),
                'type': 'sell',
                'qty': shares_sold,
                'px': px,
                'amount': amount,
                'cum_qty': total_shares,
                'market_value': px * total_shares            
            })

    sp500_df = pd.DataFrame(transactions)
    return sp500_df

def calculate_irr(df: pd.DataFrame) -> float:
    returns = pyxirr.xirr(dict(zip(df['date'], df['amount'])))
    assert returns is not None
    return returns * 100

def total_investment(df: pd.DataFrame) -> float:
    total = df[df['desc'] != 'current value']['amount'].sum() * -1
    return total

def current_value(df: pd.DataFrame) -> float:
    return df[df['desc'] == 'current value']['amount'].sum()

def investment_summary() -> list[pd.DataFrame]:
    my_df = calculate_my_returns()
    sp500_df = calculate_sp500_returns(my_df)

    total_invest = total_investment(my_df)
    market_value = current_value(my_df)
    pnl = market_value - total_invest
    pnl_perc = pnl / total_invest * 100

    my_irr = calculate_irr(my_df.groupby('date').agg({'amount': 'sum', 'cumulative_investment': 'last'}).reset_index())
    sp500_irr = calculate_irr(sp500_df.groupby('date').agg({'amount': 'sum', 'market_value': 'last'}).reset_index())

    sp500_equivalent_market_value = sp500_df[sp500_df['type'] == 'current value'].loc[:, 'market_value'].iloc[0]
    sp500_equivalent_shares = sp500_df[sp500_df['type'] == 'current value'].loc[:, 'cum_qty'].iloc[0]
    beat_sp500_perc = (market_value - sp500_equivalent_market_value) / sp500_equivalent_market_value * 100

    print(f"Total investment: S$ {total_invest:,.0f}")
    print(f"Market value: S$ {market_value:,.0f}")
    print(f"PnL: S$ {pnl:,.2f}")
    print(f"Returns: {pnl_perc:.2f}%")
    print(f"IRR of my investments: {my_irr:.2f}%")
    print(f"IRR of sp500: {sp500_irr:.2f}%")

    print(f"S&P500 equivalent shares: {sp500_equivalent_shares:.2f}")
    print(f"S&P500 equivalent market value: S$ {sp500_equivalent_market_value:,.0f}")
    print(f"beat S&P500 by: {beat_sp500_perc:.2f}%")

    return [my_df, sp500_df]

def plot_investment_comparison(my_df: pd.DataFrame, sp500_df: pd.DataFrame):
    start_date = sp500_df['date'].min()
    end_date = my_df[my_df['desc'] == 'current value']['date'].iloc[0]
    sp500_prices = get_sp500_data(start_date, end_date)

    daily_values = pd.DataFrame(index=sp500_prices.index)
    daily_values['qty'] = np.nan

    # Calculate daily S&P500 shares and values
    for date in daily_values.index:
        # Get cumulative shares up to this date
        mask = sp500_df['date'] <= date
        if mask.any():
            daily_values.loc[date, 'qty'] = sp500_df[mask]['cum_qty'].iloc[-1]
        else:
            daily_values.loc[date, 'qty'] = 0

    daily_values['sp500_close'] = sp500_prices['Close']
    daily_values['sp500_value'] = daily_values['qty'] * daily_values['sp500_close']

    # Get final values
    final_portfolio_value = my_df[my_df['desc'] == 'current value']['amount'].iloc[0]
    final_sp500_value = sp500_df[sp500_df['type'] == 'current value']['amount'].iloc[0]

    # Plot
    import plotly.graph_objects as go
    fig = go.Figure()

    # Plot cumulative investment line (green)
    fig.add_trace(go.Scatter(
        x=my_df['date'],
        y=my_df['cumulative_investment'],
        mode='lines',
        name='Total Invested (S$)',
        line=dict(color='green')
    ))

    # Plot S&P500 mark-to-market value (red)
    fig.add_trace(go.Scatter(
        x=daily_values.index,
        y=daily_values['sp500_value'],
        mode='lines',
        name='S&P500 Mark-to-Market (S$)',
        line=dict(color='red')
    ))

    # Add marker for actual portfolio value (blue)
    fig.add_trace(go.Scatter(
        x=[end_date],
        y=[final_portfolio_value],
        mode='markers+text',
        name='Current Portfolio Value',
        text=[f'Portfolio: S${final_portfolio_value:,.0f}'],
        textposition='top right',
        marker=dict(color='blue', size=10)
    ))

    # Add marker for S&P500 final value (red)
    fig.add_trace(go.Scatter(
        x=[end_date],
        y=[final_sp500_value],
        mode='markers+text',
        name='S&P500 Final Value',
        text=[f'S&P500: S${final_sp500_value:,.0f}'],
        textposition='bottom right',
        marker=dict(color='red', size=10)
    ))

    fig.update_layout(
        title='Investment Growth Comparison',
        xaxis_title='Date',
        yaxis_title='Value (S$)',
        showlegend=True
    )
    
    fig.show()

# Enum for sides
class Side(Enum):
    BUY = "buy"
    SELL = "sell"


class Holdings: 
    def __init__(self, ticker: str):
        self.ticker = ticker
        self.shares = 0.0
        self.sold_shares = 0.0
        self.bought_shares = 0.0
        self.market_value = 0.0
        self.market_px = get_last_close_px(self.ticker)
        self.investment = 0.0           # total buy cash outflow
        self.proceeds = 0.0        # total sell cash inflow
        self.remaining_cost_basis = 0.0  # cost of remaining open shares
        self.cost_basis = 0.0
        self.cost_basis_per_share = 0.0
        self.realised_pnl = 0.0
        self.unrealised_pnl = 0.0
        self.pnl = 0.0
        self.returns = 0.0

    def add_transaction(self, shares: float, px: float, side: Side):
        if side == Side.SELL:
            # average-cost method
            avg_cost = (self.remaining_cost_basis / self.shares) if self.shares > 0 else 0.0
            cost_sold = avg_cost * shares
            proceeds = shares * px

            self.shares -= shares
            self.sold_shares += shares
            self.proceeds += proceeds

            self.realised_pnl += proceeds - cost_sold
            self.remaining_cost_basis -= cost_sold
        else:
            # BUY
            cost = shares * px
            self.shares += shares
            self.bought_shares += shares
            self.investment += cost
            self.remaining_cost_basis += cost

        # Recompute mark-to-market
        self.cost_basis = self.remaining_cost_basis
        self.cost_basis_per_share = (self.remaining_cost_basis / self.shares) if self.shares > 0 else 0.0
        self.market_value = self.market_px * self.shares
        self.unrealised_pnl = (self.market_px - self.cost_basis_per_share) * self.shares if self.shares > 0 else 0.0
        self.pnl = self.realised_pnl + self.unrealised_pnl

        # Returns: for open positions use remaining cost basis; for closed, use realised over total investment
        if self.shares > 0 and self.cost_basis > 0:
            self.returns = (self.pnl / self.cost_basis) * 100
        elif self.shares == 0 and self.investment > 0:
            self.returns = (self.realised_pnl / self.investment) * 100
        else:
            self.returns = 0.0


def get_last_close_px(ticker: str) -> float:
    data = yf.download(ticker, period='1d', interval='1d', progress=False, auto_adjust=True)
    assert data is not None, f"No data found for ticker: {ticker}"
    if data.empty:
        raise ValueError(f"No data found for ticker: {ticker}")
    
    return data['Close'].iloc[-1].item()


def get_daily_change_pct(ticker: str) -> float:
    # Fetch a wider window to ensure we have at least two trading days
    data = yf.download(ticker, period='7d', interval='1d', progress=False, auto_adjust=True)
    assert data is not None, f"No data found for ticker: {ticker}"
    if data.empty:
        return np.nan
    
    closes = data['Close'].dropna()
    if len(closes) < 2:
        return np.nan
    
    last_close = closes.iloc[-1].item()
    previous_close = closes.iloc[-2].item()
    
    if previous_close == 0:
        return np.nan
    return ((last_close - previous_close) / previous_close) * 100


def get_usdsgd_rate() -> float:
    return get_last_close_px('USDSGD=X')


def find_sum_of_columns(df):
    sum_format = {
        'ticker': ['Total'],
        'native_symbol': [np.nan],
        'shares': [np.nan],
        'investment': [df['investment'].sum()],
        'cost_basis': [df['cost_basis'].sum()],
        'cost_basis_per_share': [np.nan],
        'market_px': [np.nan],
        'market_value': [df['market_value'].sum()],
        'realised_pnl': [df['realised_pnl'].sum()],
        'unrealised_pnl': [df['unrealised_pnl'].sum()],
        'pnl': [df['pnl'].sum()],
        'returns': [(df['pnl'].sum() / df['cost_basis'].sum()) * 100],
        'daily_change_pct': [np.nan]
    }
    dividend_cols = [col for col in df.columns if col.startswith('dividends_')]
    for col in dividend_cols:
        sum_format[col] = [df[col].sum()]

    totals = pd.DataFrame(sum_format)
    df = pd.concat([df, totals], ignore_index=True)
    return df


def add_daily_change_pct(df: pd.DataFrame) -> pd.DataFrame:
    def safe_change(ticker):
        if not isinstance(ticker, str):
            return np.nan
        if ticker.strip().lower() == 'total':
            return np.nan
        try:
            return get_daily_change_pct(ticker)
        except Exception:
            return np.nan
    df['daily_change_pct'] = df['ticker'].apply(safe_change)
    return df


def add_dividends_to_holdings(df: pd.DataFrame, years: list[int]) -> pd.DataFrame:
    usdsgd_rate = get_usdsgd_rate()  # Get conversion rate once
    
    for ticker in df['ticker'].unique():
        if not isinstance(ticker, str) or ticker.strip().lower() == 'total':
            continue
        stock = yf.Ticker(ticker)
        dividends = stock.dividends
        if dividends.empty:
            print(f"No dividend data for {ticker}")
            continue
        
        # Determine currency based on ticker suffix
        is_sgd = ticker.endswith('.SI')  # Singapore stocks end with .SI
        
        dividends.index = pd.to_datetime(dividends.index, errors='coerce')
        
        for year in years:
            year_mask = dividends.index.to_series().dt.year == year
            year_dividends = dividends[year_mask]
            if year_dividends.empty:
                print(f"No dividends for {ticker} in {year}")
                continue

            total_dividend = year_dividends.sum()
            shares = df.loc[df['ticker'] == ticker, 'shares']
            
            # Convert to SGD if necessary
            dividend_sgd = total_dividend if is_sgd else total_dividend * usdsgd_rate
            df.loc[df['ticker'] == ticker, f'dividends_{year}'] = round(dividend_sgd * shares, 2)
    
    return df


def calculate_stock_holdings(df: pd.DataFrame, sort_col: str) -> pd.DataFrame:
    holdings = {ticker: Holdings(ticker) for ticker in df['ticker'].unique()}
    for _, row in df.iterrows():
        ticker = row['ticker']
        holdings[ticker].add_transaction(row['qty'], row['px'], Side(row['side'].lower()))

    usdsgd_rate = get_usdsgd_rate()

    rows = []
    for h in holdings.values():
        is_sgd = h.ticker.endswith('.SI')
        sym = 'S$' if is_sgd else '$'
        fx = 1.0 if is_sgd else usdsgd_rate
        rows.append({
            'ticker': h.ticker,
            'native_symbol': sym,
            'shares': h.shares,
            # native currency for per-share and price
            'cost_basis_per_share': round(h.cost_basis_per_share, 2),
            'market_px': round(h.market_px, 2),
            # SGD for aggregate monetary
            'cost_basis': round(h.cost_basis * fx, 2),
            'investment': round(h.investment * fx, 2),
            'market_value': round(h.market_px * h.shares * fx, 2),
            'realised_pnl': round(h.realised_pnl * fx, 2),
            'unrealised_pnl': round(h.unrealised_pnl * fx, 2),
            'pnl': round(h.pnl * fx, 2),
            'returns': round(h.returns, 2)
        })

    out = pd.DataFrame(rows)
    out.sort_values(by=sort_col, ascending=False, inplace=True)
    out.reset_index(drop=True, inplace=True)
    out = add_daily_change_pct(out)
    out = add_dividends_to_holdings(out, [2024, 2025])
    return find_sum_of_columns(out)


def pretty_print(df: pd.DataFrame):
    def color_negative(val):
        if pd.isna(val):
            return ''
        try:
            value = float(str(val).strip('%').replace(',', ''))
            color = 'red' if value < 0 else 'green'
            return f'background-color: {color}; color: white'
        except Exception:
            return ''

    # Build a display copy with per-row currency symbols for native columns
    disp = df.copy()
    def fmt_native(row, col):
        val = row[col]
        if pd.isna(val):
            return ''
        sym = row.get('native_symbol', '')
        return f"{sym} {val:,.2f}" if isinstance(val, (int, float, np.floating)) else str(val)

    if 'native_symbol' in disp.columns:
        disp['market_px'] = disp.apply(lambda r: fmt_native(r, 'market_px'), axis=1)
        disp['cost_basis_per_share'] = disp.apply(lambda r: fmt_native(r, 'cost_basis_per_share'), axis=1)

    format_dict = {
        'shares': '{:,.0f}',
        'investment': 'S$ {:,.2f}',
        'cost_basis': 'S$ {:,.2f}',
        'market_value': 'S$ {:,.2f}',
        'realised_pnl': 'S$ {:,.2f}',
        'unrealised_pnl': 'S$ {:,.2f}',
        'pnl': 'S$ {:,.2f}',
        'returns': '{:,.2f} %',
        'daily_change_pct': '{:,.2f} %'
    }
    dividends_cols = [col for col in disp.columns if col.startswith('dividends_')]
    for col in dividends_cols:
        format_dict[col] = 'S$ {:,.2f}'

    # Order columns nicely
    col_order = ['ticker', 'shares', 'market_px', 'cost_basis_per_share', 'investment',
                 'cost_basis', 'market_value', 'realised_pnl', 'unrealised_pnl', 'pnl',
                 'returns', 'daily_change_pct'] + dividends_cols
    disp = disp[[c for c in col_order if c in disp.columns]]

    return disp.style.format(format_dict, na_rep="").set_properties(**{
        'text-align': 'right',
        'padding': '5px'
    }).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]},
        {'selector': '', 'props': [('border', '1px solid #ddd')]},
        {'selector': 'tbody tr:last-child', 'props': [('font-weight', 'bold'), ('border-top', '2px solid black')]}
    ]).map(color_negative, subset=['daily_change_pct', 'returns'])

In [ ]:
my_df, sp500_df = investment_summary()

In [ ]:
plot_investment_comparison(my_df, sp500_df)

In [ ]:
stocks = pd.read_excel(STOCK_FILE, sheet_name="stocks")
stocks = parse_timestamp( stocks )
stocks

In [ ]:
stocks[stocks['ticker'] == 'BABA']

In [ ]:
stock_returns = calculate_stock_holdings(stocks, "market_value")
stock_returns = add_dividends_to_holdings(stock_returns, [2024, 2025])
pretty_print(stock_returns)